In [1]:
%load_ext autoreload
%autoreload 2

In [3]:
import warnings
warnings.filterwarnings('ignore',category=FutureWarning)
import os
import sys
sys.path.insert(1, 'model/')
os.environ["TF_CPP_MIN_LOG_LEVEL"] = "2"
from model.FINDER_API import FINDER_API
from py_utils.FINDER_test_utils import *
import os
import tqdm
import numpy as np
import wandb
os.environ["WANDB_MODE"] = 'disabled'


# Specify and load model

In [4]:
# model_path =  'best_models/tsp_2d/nrange_15_20_len_1011194/'
# model_path =  'best_models/tsp_2d/nrange_15_20_len_1011362/'
# model_path =  'best_models/tsp_2d/nrange_15_20_len_1009797/'
# model_path = 'best_models/tsp_2d/nrange_15_20_len_1011914/'

# insert function which automatically searches for best model with specified name
model_name = 'AGNN_default_len_10113392076640768'
model_name = 'AGNN_RNN_len_10117276006490983'
model_name = 'AGNN_no_helperfunction_len_10488634773857972'
model_name = 'AGNN_no_help_only_start_end_+_edge_prob_2021-09-01_08:31:56_len_100233'
model_name = 'test_2021-09-16_10:49:03_len_100258'
model_name = 'AGNN_no_help_only_start_end_+_edge_prob_+_unfixed_start_2021-09-13_12:10:11_len_100131'
model_name = 'model_2021-09-23_15:11:10_len_100987'

model_path = f'saved_models/tsp_2d/nrange_20_20/{model_name}'

model_name = 'standard_2021-10-18_02:36:38_len_100324'
model_path = f'test_models/{model_name}'

config_path = f'{model_path}/config.txt'
api = FINDER_API(config_path=config_path)

# get best checkpoint path
best_ckpt_file_path = get_best_ckpt(model_path)
print(best_ckpt_file_path)
# load checkpoint into Finder
api.load_model(ckpt_path=best_ckpt_file_path)

Error when loading key 'sample_steps' from external config file!
Using default value 64 instead!


Failed to detect the name of this notebook, you can set it manually with the WANDB_NOTEBOOK_NAME environment variable to enable code saving.


Gpu available: True
Built with cuda: True


wandb: Currently logged in as: dschaub (use `wandb login --relogin` to force relogin)


Current config: {'help_func': 0, 'reward_normalization': 'max', 'reward_sign': -1, 'fix_start_node': 0, 'net_type': 'AGNN', 'max_bp_iter': 3, 'aggregatorID': 1, 'node_init_dim': 6, 'edge_init_dim': 4, 'state_init_dim': 1, 'node_embed_dim': 64, 'edge_embed_dim': 64, 'state_embed_dim': 128, 'embeddingMethod': 3, 'ignore_covered_edges': 1, 'selected_nodes_inclusion': 1, 'focus_start_end_node': 1, 'state_representation': 1, 'IsHuberloss': 0, 'BATCH_SIZE': 64, 'initialization_stddev': 0.01, 'MAX_ITERATION': 100000, 'LEARNING_RATE': 0.001, 'Alpha': 0.001, 'save_interval': 300, 'num_env': 1, 'dropout_rate': 0.0, 'g_type': 'tsp_2d', 'NUM_MIN': 20, 'NUM_MAX': 20, 'NN_ratio': 1.0, 'n_generator': 1000, 'train_path': 'data/train_sets/synthetic_n_20_200000', 'train_scale_fac': 1e-06, 'decoder_type': 0, 'REG_HIDDEN': 32, 'search_strategy': 'greedy', 'beam_width': 64, 'sample_steps': 64, 'test_batch_size': 512, 'IsDoubleDQN': 0, 'N_STEP': 5, 'GAMMA': 1.0, 'UPDATE_TIME': 1000, 'eps_start': 1.0, 'eps_e

# Specify test sets

In [7]:
# data_dir = 'test_sets/synthetic_nrange_10_20_1000/'
# data_dir = 'test_sets/synthetic_nrange_15_20_1000/'
# data_dir = 'test_sets/synthetic_nrange_40_50_1000/'
# data_dir = 'valid_sets/synthetic_nrange_10_20_200/'
# data_dir = 'valid_sets/synthetic_nrange_15_20_200/'
# data_dir = 'data/test_sets/tsp_min-n=15_max-n=20_num-graph=1000_type=random/'
# data_dir_0 = 'data/test_sets/tsp_min-n=15_max-n=20_num-graph=1000_type=random/'
# data_dir_1 = 'data/test_sets/tsp_min-n=40_max-n=50_num-graph=1000_type=random/'
# data_dir_2 = 'data/test_sets/tsp_min-n=50_max-n=100_num-graph=1000_type=random/'
# data_dir_3 = 'data/test_sets/tsp_min-n=100_max-n=200_num-graph=1000_type=random/'
data_dir_valid = 'data/valid_sets/synthetic_nrange_20_20_100'
data_dir_0 = 'data/test_sets/synthetic_n_20_1000'
data_dir_1 = 'data/test_sets/synthetic_n_50_1000'
test_dirs = [data_dir_0, data_dir_1]

# Run tests

## Run standard tests

In [5]:
search_strategy = 'greedy'

In [8]:
# sanity check
lengths, solutions, sol_times = api.run_test(data_dir_valid, scale_factor=0.000001)
approx_ratios, mean_approx_ratio = get_approx_ratios(data_dir_valid, lengths)
print(mean_approx_ratio)

Loading test graphs...


100%|██████████| 20/20 [00:02<00:00,  7.72it/s]


1.0032381307756169


In [9]:
mean_approx_ratios = []
for data_dir in test_dirs[:1]:
    # run test
    lengths, solutions, sol_times = api.run_test(data_dir, scale_factor=0.000001)
    approx_ratios, mean_approx_ratio = get_approx_ratios(data_dir, lengths)
    mean_approx_ratios.append(mean_approx_ratio)
    print(mean_approx_ratio)
    # save test results
    save_approx_ratios(data_dir, approx_ratios, model_name, suffix=search_strategy)
    save_solutions(data_dir, solutions, model_name, suffix=search_strategy)
    save_lengths(data_dir, lengths, model_name, suffix=search_strategy)

Loading test graphs...


100%|██████████| 20/20 [00:19<00:00,  1.02it/s]


1.008371726664478
Saving approximation ratios...


100%|██████████| 1000/1000 [00:08<00:00, 113.40it/s]


Saving solutions...


100%|██████████| 1000/1000 [00:09<00:00, 109.37it/s]


Saving solution lengths...


100%|██████████| 1000/1000 [00:08<00:00, 114.27it/s]


# Beam search


In [12]:
# specify different configurations
beam_widths = [8, 16, 32, 64]
beam_widths = [1, 128]
# use beam search during testing
search_strategy = 'beam_search+'
api.DQN.cfg['search_strategy'] = search_strategy
for beam_width in beam_widths:
    suffix = f'{search_strategy}_{beam_width}'
    api.DQN.cfg['beam_width'] = beam_width
    mean_approx_ratios = []
    for data_dir in test_dirs[0:1]:
        # run test
        lengths, solutions, sol_times = api.run_test(data_dir, scale_factor=0.000001)
        approx_ratios, mean_approx_ratio = get_approx_ratios(data_dir, lengths)
        mean_approx_ratios.append(mean_approx_ratio)
        print("Approximation ratio:", mean_approx_ratio)
        # save test results
        save_approx_ratios(data_dir, approx_ratios, model_name, suffix=suffix)
        save_solutions(data_dir, solutions, model_name, suffix=suffix)
        save_lengths(data_dir, lengths, model_name, suffix=suffix)

Loading test graphs...


100%|██████████| 1000/1000 [02:22<00:00,  7.03it/s]


Approximation ratio: 1.008371726664478
Saving approximation ratios...


100%|██████████| 1000/1000 [00:07<00:00, 131.10it/s]


Saving solutions...


...

100%|██████████| 1000/1000 [00:07<00:00, 135.88it/s]


Saving solutions...


100%|██████████| 1000/1000 [00:07<00:00, 134.61it/s]


Saving solution lengths...


100%|██████████| 1000/1000 [00:07<00:00, 137.75it/s]


In [7]:
# use beam search during testing
search_strategy = 'beam_search+'
beam_width = 128
batch_size = 128

api.DQN.cfg['search_strategy'] = search_strategy
api.DQN.cfg['beam_width'] = beam_width
api.DQN.cfg['BATCH_SIZE'] = batch_size

suffix = f'{search_strategy}_{beam_width}'

In [8]:
# sanity check
lengths, solutions, sol_times = api.run_test(data_dir_valid, scale_factor=0.000001)
approx_ratios, mean_approx_ratio = get_approx_ratios(data_dir_valid, lengths)
print(mean_approx_ratio)

{'help_func': 0, 'reward_normalization': 'max', 'reward_sign': -1, 'fix_start_node': 0, 'net_type': 'AGNN', 'max_bp_iter': 3, 'aggregatorID': 1, 'node_init_dim': 6, 'edge_init_dim': 4, 'state_init_dim': 1, 'node_embed_dim': 64, 'edge_embed_dim': 64, 'state_embed_dim': 192, 'embeddingMethod': 3, 'ignore_covered_edges': 0, 'selected_nodes_inclusion': 1, 'focus_start_end_node': 1, 'state_representation': 0, 'IsHuberloss': 0, 'BATCH_SIZE': 128, 'initialization_stddev': 0.01, 'MAX_ITERATION': 1000000, 'LEARNING_RATE': 0.001, 'Alpha': 0.001, 'save_interval': 300, 'num_env': 1, 'dropout_rate': 0.0, 'g_type': 'tsp_2d', 'NUM_MIN': 20, 'NUM_MAX': 20, 'NN_ratio': 1.0, 'n_generator': 1000, 'train_path': 'data/train_sets/synthetic_n_20_200000', 'train_scale_fac': 1e-06, 'decoder_type': 0, 'REG_HIDDEN': 32, 'search_strategy': 'beam_search+', 'beam_width': 128, 'IsDoubleDQN': 0, 'N_STEP': 5, 'GAMMA': 1.0, 'UPDATE_TIME': 1000, 'eps_start': 1.0, 'eps_end': 0.05, 'eps_step': 10000.0, 'MEMORY_SIZE': 1500

100%|██████████| 100/100 [06:01<00:00,  3.62s/it]

1.0000735294679544


In [11]:
mean_approx_ratios = []
for data_dir in test_dirs[0:1]:
    # run test
    lengths, solutions, sol_times = api.run_test(data_dir, scale_factor=0.000001)
    approx_ratios, mean_approx_ratio = get_approx_ratios(data_dir, lengths)
    mean_approx_ratios.append(mean_approx_ratio)
    print(mean_approx_ratio)
    # save test results
    save_approx_ratios(data_dir, approx_ratios, model_name, suffix=suffix)
    save_solutions(data_dir, solutions, model_name, suffix=suffix)
    save_lengths(data_dir, lengths, model_name, suffix=suffix)

{'help_func': 0, 'reward_normalization': 'max', 'reward_sign': -1, 'fix_start_node': 0, 'net_type': 'AGNN', 'max_bp_iter': 3, 'aggregatorID': 1, 'node_init_dim': 6, 'edge_init_dim': 4, 'state_init_dim': 1, 'node_embed_dim': 64, 'edge_embed_dim': 64, 'state_embed_dim': 192, 'embeddingMethod': 3, 'ignore_covered_edges': 0, 'selected_nodes_inclusion': 1, 'focus_start_end_node': 1, 'state_representation': 0, 'IsHuberloss': 0, 'BATCH_SIZE': 128, 'initialization_stddev': 0.01, 'MAX_ITERATION': 1000000, 'LEARNING_RATE': 0.001, 'Alpha': 0.001, 'save_interval': 300, 'num_env': 1, 'dropout_rate': 0.0, 'g_type': 'tsp_2d', 'NUM_MIN': 20, 'NUM_MAX': 20, 'NN_ratio': 1.0, 'n_generator': 1000, 'train_path': 'data/train_sets/synthetic_n_20_200000', 'train_scale_fac': 1e-06, 'decoder_type': 0, 'REG_HIDDEN': 32, 'search_strategy': 'beam_search+', 'beam_width': 128, 'IsDoubleDQN': 0, 'N_STEP': 5, 'GAMMA': 1.0, 'UPDATE_TIME': 1000, 'eps_start': 1.0, 'eps_end': 0.05, 'eps_step': 10000.0, 'MEMORY_SIZE': 1500

100%|██████████| 1000/1000 [58:57<00:00,  3.54s/it]


1.0000898666332352
Saving approximation ratios...


100%|██████████| 1000/1000 [00:08<00:00, 118.84it/s]


Saving solutions...


100%|██████████| 1000/1000 [00:08<00:00, 117.06it/s]


Saving solution lengths...


100%|██████████| 1000/1000 [00:08<00:00, 118.30it/s]


# TSP sampling/augmentation

In [11]:
# specify different configurations
sample_steps = [2, 4, 8, 16, 32]
# use beam search during testing
search_strategy = 'sampling'
api.DQN.cfg['search_strategy'] = search_strategy
for sample_step in sample_steps:
    suffix = f'{search_strategy}_{sample_step}'
    api.DQN.cfg['sample_steps'] = sample_step
    mean_approx_ratios = []
    for data_dir in test_dirs[0:1]:
        # run test
        lengths, solutions, sol_times = api.run_test(data_dir, scale_factor=0.000001)
        approx_ratios, mean_approx_ratio = get_approx_ratios(data_dir, lengths)
        mean_approx_ratios.append(mean_approx_ratio)
        print("Approximation ratio:", mean_approx_ratio)
        # save test results
        save_approx_ratios(data_dir, approx_ratios, model_name, suffix=suffix)
        save_solutions(data_dir, solutions, model_name, suffix=suffix)
        save_lengths(data_dir, lengths, model_name, suffix=suffix)

Loading test graphs...


100%|██████████| 1/1 [00:53<00:00, 53.99s/it]


Approximation ratio: 1.0040125489287055
Saving approximation ratios...


100%|██████████| 1000/1000 [00:09<00:00, 107.94it/s]


Saving solutions...


...

100%|██████████| 1000/1000 [00:08<00:00, 119.88it/s]


Saving solutions...


100%|██████████| 1000/1000 [00:08<00:00, 116.21it/s]


Saving solution lengths...


100%|██████████| 1000/1000 [00:08<00:00, 119.19it/s]


# Test helper function

In [ ]:
graph = graph_list[0]
length, solution, time_ = dqn.Evaluate(g=graph)

In [ ]:
solution, length

In [ ]:
# check whether length calculation is correct
def calc_tour_length(graph, solution):
    tot_len = 0
    for i in range(np.array(solution).shape[0]):
        if i == np.array(solution).shape[0] - 1:
            tot_len += graph[solution[i]][solution[0]]['weight']
        else:
            tot_len += graph[solution[i]][solution[i + 1]]['weight']
    return tot_len

In [ ]:
def add_node(graph, cur_sol, new_node):
    cur_dist = 10000000.0
    for i in range(0, np.array(cur_sol).shape[0]):
        if i + 1 == np.array(cur_sol).shape[0]:
            adj = cur_sol[0]
        else:
            adj = cur_sol[i + 1]
        if cur_sol[i] == adj:
            cost = graph[new_node][cur_sol[i]]['weight'] + graph[new_node][adj]['weight']
        else:
            cost = graph[new_node][cur_sol[i]]['weight'] + graph[new_node][adj]['weight'] - graph[cur_sol[i]][adj]['weight']
        if cost < cur_dist:
            cur_dist = cost
            pos = i + 1
    cur_sol.insert(pos, new_node)
    return  cur_dist

In [ ]:
def calc_helper_tour(g):
    cur_sol = [0]
    rem_nodes = list(np.arange(g.number_of_nodes()))
    rem_nodes.remove(0)
    tour_length = 0
    while np.array(cur_sol).shape[0] < g.number_of_nodes():
        if np.array(rem_nodes).shape[0]-1 == 0:
            rand_idx = 0
        else:
            rand_idx = np.random.randint(0, np.array(rem_nodes).shape[0]-1)
        new_node = rem_nodes[rand_idx]
        tour_length += add_node(g, cur_sol, new_node)
        rem_nodes.remove(new_node)
    return tour_length, cur_sol


In [ ]:
max_result = -np.inf
min_result = np.inf
for i in range(100):
    help_lengths = []
    for graph in graph_list:
        help_lengths.append(calc_helper_tour(graph)[0])

    true_lengths = None
    with open(data_dir+'lengths.txt', 'r') as f:
        lines = f.readlines()
        lines = [float(line.split(':')[-1].strip()) for line in lines]
    true_lengths = lines
    result = np.mean([length[0]/length[1] for length in zip(help_lengths, true_lengths)])
    if result < min_result:
        min_result = result
    if result > max_result:
        max_result = result

In [ ]:
print(min_result)
print(max_result)